## LTSM

## BERTimbau

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_plain = pd.read_csv('../data/enc_tok_nostopword_nopunct_lemm.csv')
df_plain.head()

,Unnamed: 0,req_text,age,age_encoded,word_tokens,lemma
0,0,quantos empregados em cada um dos atuais níve...,a2,1,"['quantos', 'empregados', 'cada', 'atuais', 'n...",quanto empregado cada atual nível salarial pca...
1,1,solicito cópia das atas do conselho de admini...,a2,1,"['solicito', 'cópia', 'atas', 'conselho', 'adm...",solicito cópia ata conselho administração bnde...
2,2,solicito informar a norma lei decreto portari...,a2,1,"['solicito', 'informar', 'norma', 'lei', 'decr...",solicito informar norma lei decreto portaria i...
3,3,solicito por gentileza a informação sobre a q...,a2,1,"['solicito', 'gentileza', 'informação', 'sobre...",solicito gentileza informação sobre quantidade...
4,4,solicito por gentileza a informação sobre a q...,a2,1,"['solicito', 'gentileza', 'informação', 'sobre...",solicito gentileza informação sobre quantidade...


In [3]:
# BERT Large
# tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
# model = AutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased')



In [4]:
from sklearn.model_selection import train_test_split


In [5]:
X = df_plain['lemma']
y = df_plain['age_encoded']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)



In [7]:
import  torch
from  transformers  import  AutoModel



In [8]:
model = AutoModel.from_pretrained('rufimelo/Legal-BERTimbau-large-TSDAE')

In [9]:
from  transformers  import  AutoTokenizer, AutoModelForMaskedLM

In [10]:
tokenizer = AutoTokenizer.from_pretrained("rufimelo/Legal-BERTimbau-large-TSDAE")

In [11]:
input_ids = tokenizer.encode('O advogado apresentou recurso para o juíz', return_tensors='pt')

with  torch.no_grad():
    outs = model(input_ids)
    encoded = outs[0][0, 1:-1]

In [14]:
embeddings = []
for text in df_plain['lemma']:
    input_ids = tokenizer(text, return_tensors="pt", max_length=512)

    with  torch.no_grad():
        outs = model(**input_ids)
        encoded = outs[0][0, 1:-1]

    embeddings.append(encoded)
 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
emb = []
for text in embeddings:
    print(text)
    aux = np.array(text, dtype=object)
    emb.append(aux)

In [ ]:
df = pd.DataFrame(emb)

In [ ]:
df.to_csv('../embeddings/BERT_newmodel.csv')

In [16]:
embeddings.to_csv()

AttributeError: 'list' object has no attribute 'to_csv'

In [ ]:
df_plain['bert'] = embeddings
df_plain

,Unnamed: 0,req_text,age,age_encoded,word_tokens,lemma,bert
0,0,quantos empregados em cada um dos atuais níve...,a2,1,"['quantos', 'empregados', 'cada', 'atuais', 'n...",quanto empregado cada atual nível salarial pca...,None
1,1,solicito cópia das atas do conselho de admini...,a2,1,"['solicito', 'cópia', 'atas', 'conselho', 'adm...",solicito cópia ata conselho administração bnde...,None
2,2,solicito informar a norma lei decreto portari...,a2,1,"['solicito', 'informar', 'norma', 'lei', 'decr...",solicito informar norma lei decreto portaria i...,None
3,3,solicito por gentileza a informação sobre a q...,a2,1,"['solicito', 'gentileza', 'informação', 'sobre...",solicito gentileza informação sobre quantidade...,None
4,4,solicito por gentileza a informação sobre a q...,a2,1,"['solicito', 'gentileza', 'informação', 'sobre...",solicito gentileza informação sobre quantidade...,None
...,...,...,...,...,...,...,...
8195,8195,desejo saber se meu nome consta na relação de ...,a4,3,"['desejo', 'saber', 'nome', 'consta', 'relação...",desejo saber nome constar relação restituição ...,None
8196,8196,desejo saber se posso utilizar o nome fantasia...,a4,3,"['desejo', 'saber', 'posso', 'utilizar', 'nome...",desejo saber poder utilizar nome fantasia sant...,None
8197,8197,desejo saber se tem e quantas vagas desocupada...,a4,3,"['desejo', 'saber', 'quantas', 'vagas', 'desoc...",desejo saber quanta vaga desocupar ufrb cargo ...,None
8198,8198,desejo saber se tenho alguma pendência na rece...,a4,3,"['desejo', 'saber', 'alguma', 'pendência', 're...",desejo saber algum pendência receitar federal ...,None


In [ ]:
df_plain['bert'].unique()

array([None], dtype=object)

In [ ]:
labels = ['emb_' + str(i) for i in range(len(embeddings[0]))]

TypeError: object of type 'NoneType' has no len()

In [ ]:
df_embeddings = pd.DataFrame(embeddings, columns=labels)
df_embeddings

In [ ]:
df_embeddings.to_csv('../embeddings/BERT_Embeddings_new.csv', index=False)

In [ ]:
# Obtenha embeddings BERTimbau
# bert_model = TFBertModel.from_pretrained("neuralmind/bert-base-portuguese-cased")
# tokenized_inputs = tokenizer(df_plain['lemma'].tolist(), return_tensors="tf", padding=True, truncation=True, max_length=512)
# bert_outputs = bert_model(tokenized_inputs)
# embeddings = bert_outputs['last_hidden_state']

In [ ]:

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_plain['age'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(embeddings.numpy(), labels, test_size=0.2, random_state=42)

AttributeError: 'list' object has no attribute 'numpy'

### Model

##

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Acurácia do modelo: {accuracy}')